In [1]:
#load packages:
import pandas as pd
import numpy as np
import config
import json
import pyreadr
import pprint

import requests
import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time
import re
import typing

#from sklearn.metrics import cohen_kappa_score, classification_report
#import krippendorff
import yaml

import src
import tqdm
import logging

#import cltrier_lib as lib
#import pyreadstat
import yaml
pd.set_option('display.max_colwidth', 100) 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [6]:
#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

#apparently the 3.1 70b model is no longer available via Trier...
MODELsmall: str = 'llama3.1:8b'
MODEL33large: str = 'llama3.3:70b' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.
MODEL33largeAzure = 'Llama-3.3-70B-Instruct' #azureml://registries/azureml-meta/models/Llama-3.3-70B-Instruct/versions/4 / options:  "data": [
MODEL31largeAzureNF = 'nf-Llama-3.1-70b-instruct'

#    {
#      "id": "gpt4o",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "gpt-4",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Llama-3.2-90B-Vision-Instruct",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "dall-e-3",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Mistral-Nemo",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Mistral-small",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Codestral-2501",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "gpt-35-turbo",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Ministral-3B",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "text-embedding-ada-002",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-4",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-4o-mini",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Mistral-Large-2411",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-text-embedding-ada-002",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-Llama-3.1-70b-instruct",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-4-vision",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "gpt-4-turbo",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-35-turbo",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Llama-3.3-70B-Instruct",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-dall-e-3",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "gpt-4-vision",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "gpt-4o-mini",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-4o",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-4o-2024-08-06",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-gpt-4-turbo",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "nf-Llama-3.1-8b-instruct",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    },
#    {
#      "id": "Llama-3.2-11B-Vision-Instruct",
#      "object": "model",
#      "created": 1677610602,
#      "owned_by": "openai"
#    }
#  ],
#  "object": "list"
#}

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
api_endpoint_embed = "https://ai-research-proxy.azurewebsites.net/embeddings"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}

In [3]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a social media comment enclosed in chevrons <..>. Identify and list the claims within this comment. Claims can be related to events, issues, opinions or concerns.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

#note updated on 30-12-2024:
SYSTEM_expansion: str = \
    """
        # Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exact target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:
    """

SYSTEM_expansion_refwords: str = \
    """
        # Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread, pay special attention to reference words like that/they/this/it, or other words that may refer to previous information like 'disagree':\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details. For example if it includes reference words, replace them with what they refer to. \n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exact target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:
    """

SYSTEM_context_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a target social media reply enclosed in chevrons <..> while taking into account the comment to which it replied (the context, provided in double chevrons <<...>>) to verbalize the claims made. Identify and list the claims within this target reply. Claims can be related to events, issues, opinions or concerns.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""


SYSTEM_claim_new: str = \
    """
        # Instruction

        Identify whether a claim adds new information (2), expands on information (1) or does not add new information (0) with respect to preceding claims. New information is present (2) if the claim introduces a new argument or perspective. 
        A claim expands on an existing argument or perspective (1) if it strongly related to it, or it provides additional context or examples to an argument or perspective in a preceding claim, or if it provides a sub-argument (parent-child relation). 
        If the claim is very similar to a preceding claim or does not add any perspective or argument, assign a value of 0. Follow these steps:


        Follow these steps:

        1. You will receive a target claim in double chevrons <<...>> along with the preceding claims in single chevrons <...>.
        2. Identify if new information is presented in the target claim with respect to the preceding posts. 
        4. If in doubt, assign a value of 0.

        Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
        Class:
	"""

SYSTEM_claim_simscore: str = \
    """
        # Instruction

        Identify degree of similarity of the information presented in a claim with respect to preceding claims in a thread on an integer scale of 1 (completely dissimilar) to 10 (identical). A score of 5 indicates that the claims share context or topic, but otherwise present different information. 
        Follow these steps:

        1. You will receive a target claim in double chevrons <<...>> along with a JSON containing the preceding claims and their claim_index enclosed in single chevrons <...>.
        2. Determine the most similar claim to the target claim in the preceding claims in terms of the information they present.. If only one preceding claim is provided pick this claim as the most similar claim.
        3. Find the claim_index of this most similar claim. If you can't decide which claim is most similar, pick a preceding comment at random.
        4. Identify the degree of similarity of the target claim with respect to that claim on a range of 1-10. 
        5. If no target claim is porvided or only '[]' return an empty list [] as value for both the most_similar_claim_index and the similarity_score, if you can't decide on the similarity score return an empty list for that value. If no preceding claims are provided or only an empty string '', return an empty list [] as value for both the most_similar_claim_index and the similarity_score.
        6. Always and only respond with the claim_index and similarity score.

        Response format in JSON:

        [
            {
                "most_similar_claim_index": "1",
                "similarity_score": "1"	
            }
        ]
	"""

SYSTEM_post_simscore: str = \
    """
        # Instruction 
        
        Identify degree of similarity of the information presented in a social media comment with respect to preceding comments in a thread on an integer scale of 1 (completely dissimilar) to 10 (identical). A score of 5 indicates that the comments share context or topic, but otherwise present different information. 
        Follow these steps:
        
        1. You will receive a target comment in double chevrons <<...>> along with a JSON containing the preceding comments and their comment_index enclosed in single chevrons <...>.
        2. Determine the most similar comment to the target comment in the preceding comments in terms of the information they present. If only one preceding comment is provided pick this comment as the most similar comment.
        3. Find the comment_index of this most similar comment. If you can't decide which comment is most similar, pick a preceding comment at random.
        4. Identify the degree of similarity of the target comment with respect to that comment on a range of 1-10. 
        5. If no target comment is provided or only '[]' return an empty list [] as value for both the most_similar_comment_index and the similarity_score, if you can't decide on the similarity score return an empty list for that value.
        6. Always and only respond with the comment_index and similarity score.
        
        Response format in JSON:
        
        [
            {
                "most_similar_comment_index": "1",
                "similarity_score": "1"	
            }
        ]
	"""

In [ ]:
#inspect TWON conversation data nesting structure:
pprint.pprint(load_json('data/TWON_export_Nils/conv_group2_august.json'))

In [10]:
#load TWON conversation data:
Nilspair = load_json('data/TWON_export_Nils/conv_group2_august.json')

In [11]:
# Add a list ID to each item
flattened_data = []
for thread_id, sublist in enumerate(Nilspair):
    for item in sublist:
        item['thread_id'] = thread_id  # Add the list ID
        flattened_data.append(item)
        
NilsThread = pd.json_normalize(flattened_data)


In [ ]:
NilsThread

In [7]:
#how many threads are there?
print(len(NilsThread['thread_id'].unique()))
#how many comments per thread on average:
print(NilsThread.groupby('thread_id').size().mean())

108644
2.0858491955377194


In [12]:
#add datetime variable:
NilsThread['parsed_create_time'] = pd.to_datetime(NilsThread['created_at'], errors='coerce', utc=True)
#add preceding  comment index variable:
NilsThread_sort = NilsThread.sort_values(by=['thread_id', 'parsed_create_time'], ascending=True)
NilsThread_sort['ascending_comment_index'] = NilsThread_sort.groupby('thread_id').cumcount()
NilsThread_sort['preceding_comment_index'] = NilsThread_sort.groupby('thread_id')['ascending_comment_index'].shift(1)
NilsThread_sort['indexcol'] = NilsThread_sort.index
NilsThread_sort.head(2)

,full_text,tweet_id,created_at,screen_name,original_user_id,retweeted_user_ID,collected_at,reply_to_id,reply_to_user,expandedURL,thread_id,parsed_create_time,ascending_comment_index,preceding_comment_index,indexcol
0,James Comer claims House Republicans were just about to crack the Hunter Biden case wide open an...,1.690720e+18,2023-08-13 13:39:27+00:00,MeidasTouch,1.243560e+18,NaN,2023-08-15 17:27:17.883106,NaN,NaN,"[{'url': 'https://t.co/ULhlQ86ycJ', 'expanded_url': 'https://www.meidastouch.com/news/inspector-...",0,2023-08-13 13:39:27+00:00,0,NaN,0
1,"@MeidasTouch Because of the Unfathomable Stupidity of the Republican Voter, GOP politicians have...",1.691029e+18,2023-08-14 10:09:35+00:00,EdHull8,8.886505e+08,NaN,2023-08-15 17:47:19.740945,1.690720e+18,1.243560e+18,[],0,2023-08-14 10:09:35+00:00,1,0.0,1


In [13]:
#select a sample of 1000 threads:
NilsThread_sample_set = NilsThread_sort.thread_id.sample(1000, random_state=42)
NilsThread1k = NilsThread_sort[NilsThread_sort['thread_id'].isin(NilsThread_sample_set)]

In [ ]:
NilsThread1k

In [11]:
#Michael Heseltine's TWON german data from JSON:
MHpost = load_json('data/TWON_sample_MHeseltine/Germany_posts_sample_Sjoerd.json')
MHreply = load_json('data/TWON_sample_MHeseltine/Germany_replies_sample_Sjoerd.json')


In [12]:
#unnest MHpost and reply:
MHpostN = pd.json_normalize(MHpost)
MHreplyN = pd.json_normalize(MHreply)
MHpostN.head(2)

,author_id,conversation_id,referenced_tweets,edit_history_tweet_ids,lang,id,text,created_at,possibly_sensitive,in_reply_to_user_id,...,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.urls,public_metrics.bookmark_count,entities.hashtags,attachments.media_keys,entities.annotations,geo.place_id
0,27245876,1623065216740106240,"[{'type': 'replied_to', 'id': '1623082761224347651'}]",[1623292019744423936],de,1623292019744423936,@ninastahr @gitta_connemann Mensch @ninastahr das wollte ich grade sagen....,2023-02-08T12:05:51.000Z,False,19530651,...,1,8,0,452,NaN,NaN,NaN,NaN,NaN,NaN
1,845282647261761536,1649771111108816898,"[{'type': 'quoted', 'id': '1649319733630566401'}]",[1649771111108816898],de,1649771111108816898,"Das sieht schon mal gut aus @energy_charts_d - zeigt auch, dass sich der erfolgreiche Kampf unsr...",2023-04-22T13:44:18.000Z,False,NaN,...,1,4,1,717,"[{'start': 234, 'end': 257, 'url': 'https://t.co/22Ys0EjhnX', 'expanded_url': 'https://twitter.c...",NaN,NaN,NaN,NaN,NaN


In [61]:
#add datetime variable to each:
# Parse datetime values
MHpostN['parsed_create_time'] = pd.to_datetime(MHpostN['created_at'], errors='coerce', utc=True)
MHreplyN['parsed_create_time'] = pd.to_datetime(MHreplyN['created_at'], errors='coerce', utc=True)
MHreplyN.head(2)


,conversation_id,in_reply_to_user_id,edit_history_tweet_ids,id,author_id,lang,text,referenced_tweets,possibly_sensitive,created_at,...,entities.urls,entities.annotations,attachments.media_keys,entities.hashtags,geo.place_id,attachments.poll_ids,entities.cashtags,withheld.country_codes,withheld.copyright,parsed_create_time
0,1609287864310702080,16808099,[1609626055735746562],1609626055735746562,1267930834490863621,de,"@Ralf_Stegner @fuecks Das wird sicher noch aufgearbeitet werden, wenn die SPD nicht mehr an der ...","[{'type': 'replied_to', 'id': '1609578597572038656'}]",False,2023-01-01T19:02:11.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01 19:02:11+00:00
1,1609287864310702080,16808099,[1609626793635467267],1609626793635467267,1328778631406104577,de,"@Ralf_Stegner @fuecks Wollen Sie doch noch einmal überlegen, für welchen Fall genau Nordstream 2...","[{'type': 'replied_to', 'id': '1609578597572038656'}]",False,2023-01-01T19:05:07.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01 19:05:07+00:00


In [13]:
NilsThread1k[:10]

,full_text,tweet_id,created_at,screen_name,original_user_id,retweeted_user_ID,collected_at,reply_to_id,reply_to_user,expandedURL,thread_id,parsed_create_time,ascending_comment_index,preceding_comment_index,indexcol
117,"If every American would get out of their partisan boxes, this is what every American SHOULD say:...",1.691070e+18,2023-08-14 12:52:42+00:00,WalshFreedom,2.364879e+08,NaN,2023-08-15 17:13:37.935350,NaN,NaN,"[{'url': 'https://t.co/ixjywZCcYV', 'expanded_url': 'https://twitter.com/i/web/status/1691070339...",57,2023-08-14 12:52:42+00:00,0,NaN,117
118,@WalshFreedom Agree with all of this!,1.691100e+18,2023-08-14 14:50:30+00:00,JMom27,2.514812e+07,NaN,2023-08-15 16:03:09.973646,1.691070e+18,2.364879e+08,[],57,2023-08-14 14:50:30+00:00,1,0.0,118
483,We’re gonna get a mugshot in Georgia aren’t we?!?!? 😈,1.691127e+18,2023-08-14 16:37:26+00:00,JoJoFromJerz,8.188931e+17,NaN,2023-08-15 17:20:21.521808,NaN,NaN,[],234,2023-08-14 16:37:26+00:00,0,NaN,483
484,@JoJoFromJerz https://t.co/iICBTy7Fn7,1.691280e+18,2023-08-15 02:43:53+00:00,bsallday1,3.028141e+09,NaN,2023-08-15 16:10:59.447257,1.691127e+18,8.188931e+17,[],234,2023-08-15 02:43:53+00:00,1,0.0,484
713,Bring back Aunt Jemima!\nBring back Uncle Ben!\nBring back the Land O’ Lakes Indian!\nBring back...,1.690911e+18,2023-08-14 02:19:15+00:00,ACTBrigitte,7.225285e+17,NaN,2023-08-15 18:06:13.350287,NaN,NaN,[],346,2023-08-14 02:19:15+00:00,0,NaN,713
714,@ACTBrigitte Cancel Bud Light! What?????,1.691207e+18,2023-08-14 21:54:13+00:00,DebbyHouse5,8.036636e+17,NaN,2023-08-15 16:12:20.432690,1.690911e+18,7.225285e+17,[],346,2023-08-14 21:54:13+00:00,1,0.0,714
884,"An extremist Supreme Court has once again reversed decades of settled law, rolled back the march...",1.674449e+18,2023-06-29 16:03:55+00:00,SenWarren,9.702073e+08,NaN,2023-08-15 17:57:58.914650,NaN,NaN,"[{'url': 'https://t.co/mUeEsQcvEY', 'expanded_url': 'https://twitter.com/i/web/status/1674448621...",429,2023-06-29 16:03:55+00:00,0,NaN,884
885,"@SenWarren So apparently you don't have faith in certain minorities, to be able to compete in th...",1.674527e+18,2023-06-29 21:16:06+00:00,Mothersdream,3.671425e+07,NaN,2023-08-15 16:18:03.248367,1.674449e+18,9.702073e+08,[],429,2023-06-29 21:16:06+00:00,1,0.0,885
1084,💜 @Theo_TJ_Jordan https://t.co/9xdBeKH0x3,1.691467e+18,2023-08-15 15:10:47+00:00,DrKarlynB,6.811832e+06,NaN,2023-08-15 16:12:19.829854,NaN,NaN,"[{'url': 'https://t.co/9xdBeKH0x3', 'expanded_url': 'https://twitter.com/theo_tj_jordan/status/1...",527,2023-08-15 15:10:47+00:00,0,NaN,1084
1085,"@DrKarlynB True words are easily spoken, my friend. 👊 I appreciate you.",1.691469e+18,2023-08-15 15:18:12+00:00,Theo_TJ_Jordan,1.328423e+18,NaN,2023-08-15 20:17:58.204249,1.691467e+18,6.811832e+06,[],527,2023-08-15 15:18:12+00:00,1,0.0,1085


In [ ]:
#test post expansion prompt creation with Llama3.3:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_id'

#sort data by Time_comment
groupeddata = NilsThread1k[:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df = df.loc[df.full_text != '',:]
    df.sort_values(by=['parsed_create_time'], ascending=True).reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.ascending_comment_index.shift(1)
    df.set_index('ascending_comment_index', inplace=True, drop=False)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            print(f'"Thread":\n<{df["full_text"][:index].to_list()}>, "Target reply":<<{row["full_text"]}>>')
            print(row['ascending_comment_index'], row['thread_id'])

100%|██████████| 5/5 [00:00<00:00, 496.83it/s]

"Thread":
<['If every American would get out of their partisan boxes, this is what every American SHOULD say:\n\n1. If there’s ever sufficient evidence Hunter Biden committed crimes, indict him. \n\n2. If there’s ever sufficient evidence Joe Biden committed crimes, indict him.\n\n3. Per our justice… https://t.co/ixjywZCcYV']>, "Target reply":<<@WalshFreedom Agree with all of this!>>
1 1 57
"Thread":
<['We’re gonna get a mugshot in Georgia aren’t we?!?!? 😈']>, "Target reply":<<@JoJoFromJerz https://t.co/iICBTy7Fn7>>
1 1 234
"Thread":
<['Bring back Aunt Jemima!\nBring back Uncle Ben!\nBring back the Land O’ Lakes Indian!\nBring back the Cleveland Indians!\nBring back Blockbuster!\nBring back the Washington Redskins!\nBring back GOYA!\n\nCancel culture has no place in America!']>, "Target reply":<<@ACTBrigitte Cancel Bud Light! What?????>>
1 1 346
"Thread":
<["An extremist Supreme Court has once again reversed decades of settled law, rolled back the march toward racial justice, and narrow

In [107]:
#test post expansion with Llama3.3:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_id'

#sort data by Time_comment
groupeddata = NilsThread1k[:500].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df = df.loc[df.full_text != '',:]
    df.sort_values(by=['parsed_create_time'], ascending=True).reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.ascending_comment_index.shift(1)
    df.set_index('ascending_comment_index', inplace=True, drop=False)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[row['indexcol'], row['ascending_comment_index'], row['thread_id'], row['full_text']]],
                    columns=['index', 'ascending_comment_index', 'thread_id', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            try: 
                chunked_result.append(
                   pd.DataFrame(
                       data=[[row['indexcol'], row['ascending_comment_index'], row['thread_id'],
                           requests.post(
                               'https://inf.cl.uni-trier.de/',
                               json={
                                   'model': MODEL33large,
                                   'system': SYSTEM_expansion,
                                   'prompt': f'"Thread":\n<{df["full_text"][:index].to_list()}>, "Target reply":<<{row["full_text"]}>>',
                                   'options': options_zero,
                                   'seed': SEED,
                                   }).json()['response']                       
                       ]],
                       columns=['index', 'ascending_comment_index', 'thread_id', 'post_expansion']
                   )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")

expanded_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_posts)

100%|██████████| 234/234 [2:49:08<00:00, 43.37s/it]  

     index  ascending_comment_index  thread_id  \
0      117                        0         57   
1      118                        1         57   
2      483                        0        234   
3      484                        1        234   
4      713                        0        346   
..     ...                      ...        ...   
495  50284                        1      24261   
496  50675                        0      24451   
497  50676                        1      24451   
498  50788                        0      24507   
499  50789                        1      24507   

                                                                                          post_expansion  
0    If every American would get out of their partisan boxes, this is what every American SHOULD say:...  
1    I completely agree that if there's sufficient evidence, Hunter Biden and Joe Biden should be ind...  
2                                                  We’re gonna get a mugshot i

In [131]:
NilsThread500merge = NilsThread1k[:500].merge(expanded_posts, left_on='indexcol', right_on='index', how='left').drop(columns=['index', 'ascending_comment_index_y', 'thread_id_y'], axis=1).rename(columns={'ascending_comment_index_x': 'ascending_comment_index', 'thread_id_x': 'thread_id'})

In [ ]:
NilsThread500merge

In [136]:
#save dataframe to parquet file:
NilsThread500merge.to_parquet('data/TWON_export_Nils/NilsThread500merge.parquet', index=False, engine='pyarrow')
expanded_posts.to_parquet('data/TWON_export_Nils/expanded_posts500.parquet', index=False, engine='pyarrow')

In [4]:
#load the parquet file:
NilsThread500merge = pd.read_parquet('data/TWON_export_Nils/NilsThread500merge.parquet', engine='pyarrow')

In [ ]:
#check whether we have the whole thread or only chains of replies of replies, in the latter case perhaps only add preceding comment rather than the whole thread as context?
#what is the post depth?
#currently the average number of tweets per thread is rather low, so I wonder whether we have the whole thread...
#MHpost probably do have the whole thread, but this corpus is in German, which can be more challenging for the model...
#also the NilsThread1k corpus does not match the structure stipulated by Nils in his email: "The repo also includes a sample dataset generated using one of my trained models. Note that the format differs from our standard structure - I've combined all tweets in each thread using the format: "\n\n>Username: {Tweet}". This approach allows me to insert the complete conversational state into the model at once.
#For each example, you'll find both the synthetic reply (generated by one of my models) and the actual user reply for comparison."
#perhaps this format is only used for the synthetic replies, and the original tweets are still in the original format?

In [138]:
#we can simply check whether the reply_to_id is always equal to the tweet_id of the preceding post
#add the tweet_id of the preceding post to the dataframe, by merging the dataframe with itself on the preceding_comment_index:
NilsThread1k_plus = NilsThread1k.merge(NilsThread1k.loc[:, ['thread_id', 'ascending_comment_index', 'tweet_id']], left_on=['thread_id','preceding_comment_index'], right_on=['thread_id','ascending_comment_index'], how='left', suffixes=('', '_preceding'))


In [142]:
(NilsThread1k_plus.loc[~NilsThread1k_plus.tweet_id_preceding.isna(),'tweet_id_preceding'] == NilsThread1k_plus.loc[~NilsThread1k_plus.tweet_id_preceding.isna(),'reply_to_id']).mean()

np.float64(1.0)

In [ ]:
#confirmation that this is a reply of reply structure, rather than a full thread

In [ ]:
#try running the expansion via the UvA Azure cloud:
#test post expansion with Llama3.3:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_id'

#sort data by Time_comment
groupeddata = NilsThread1k[:500].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df = df.loc[df.full_text != '',:]
    df.sort_values(by=['parsed_create_time'], ascending=True).reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.ascending_comment_index.shift(1)
    df.set_index('ascending_comment_index', inplace=True, drop=False)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[row['indexcol'], row['ascending_comment_index'], row['thread_id'], row['full_text']]],
                    columns=['index', 'ascending_comment_index', 'thread_id', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            retry_count = 0
            max_retries = 10
           
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODEL33largeAzure,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": SYSTEM_expansion
                                    },
                                    {
                                        "role": "user",
                                        "content": f'"Thread":\n<{df["full_text"][:index].to_list()}>, "Target reply":<<{row["full_text"]}>>',
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                
                            }
                        )  
                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[row['indexcol'], row['ascending_comment_index'], row['thread_id'], data_response["choices"][0]["message"]["content"]]],                                
                            columns=['index', 'ascending_comment_index', 'thread_id', 'post_expansion']
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        retry_after = response.headers.get("Retry-After")
                        error_message = response.json().get("error", {}).get("message", "")
                        retry_after = 30  # Default to 30 seconds if not found

                        # Extract retry time from the error message
                        if "Try again in" in error_message:
                            match = re.search(r"Try again in (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                        elif "Please retry after" in error_message:
                            match = re.search(r"Please retry after (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                                
                        else:
                            retry_after = 30  # Default to 30 seconds if not found
                            print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")

                        print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                        time.sleep(retry_after)

                    #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    #    print(response.text)
                    #    time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 

expanded_X500_posts_Azure33 = pd.concat(chunked_result, ignore_index=True)
print(expanded_X500_posts_Azure33)


  0%|          | 0/234 [00:00<?, ?it/s]

  5%|▍         | 11/234 [00:09<05:03,  1.36s/it]

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400,\"innererror\":{\"code\":\"ResponsibleAIPolicyViolation\",\"content_filter_result\":{\"hate\":{\"filtered\":true,\"severity\":\"medium\"},\"self_harm\":{\"filtered\":false,\"severity\":\"safe\"},\"sexual\":{\"filtered\":false,\"severity\":\"safe\"},\"violence\":{\"filtered\":false,\"severity\":\"safe\"}}}}}. Received Model Group=Llama-3.3-70B-Instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


 12%|█▏        | 29/234 [00:22<02:02,  1.67it/s]

Failed to connect to API. Status code: 500. Retrying in 20 seconds...
{"error":{"message":"litellm.APIConnectionError: APIConnectionError: Azure_aiException - Invalid response object Traceback (most recent call last):\n  File \"/usr/lib/python3.13/site-packages/litellm/litellm_core_utils/llm_response_utils/convert_dict_to_response.py\", line 473, in convert_to_model_response_object\n    reasoning_content, content = _extract_reasoning_content(\n                                 ~~~~~~~~~~~~~~~~~~~~~~~~~~^\n        choice[\"message\"]\n        ^^^^^^^^^^^^^^^^^\n    )\n    ^\n  File \"/usr/lib/python3.13/site-packages/litellm/litellm_core_utils/llm_response_utils/convert_dict_to_response.py\", line 253, in _extract_reasoning_content\n    return message[\"reasoning_content\"], message[\"content\"]\n                                         ~~~~~~~^^^^^^^^^^^\nKeyError: 'content'\n\n\nreceived_args={'response_object': {'choices': [{'content_filter_results': {'hate': {'filtered': False, 'seve

...

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400,\"innererror\":{\"code\":\"ResponsibleAIPolicyViolation\",\"content_filter_result\":{\"hate\":{\"filtered\":false,\"severity\":\"safe\"},\"self_harm\":{\"filtered\":false,\"severity\":\"safe\"},\"sexual\":{\"filtered\":false,\"severity\":\"safe\"},\"violence\":{\"filtered\":true,\"severity\":\"medium\"}}}}}. Received Model Group=Llama-3.3-70B-Instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


 94%|█████████▎| 219/234 [06:47<00:22,  1.53s/it]

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400,\"innererror\":{\"code\":\"ResponsibleAIPolicyViolation\",\"content_filter_result\":{\"hate\":{\"filtered\":true,\"severity\":\"medium\"},\"self_harm\":{\"filtered\":false,\"severity\":\"safe\"},\"sexual\":{\"filtered\":false,\"severity\":\"safe\"},\"violence\":{\"filtered\":false,\"severity\":\"safe\"}}}}}. Received Model Group=Llama-3.3-70B-Instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


100%|██████████| 234/234 [06:57<00:00,  1.78s/it]

     index  ascending_comment_index  thread_id  \
0      117                        0         57   
1      118                        1         57   
2      483                        0        234   
3      484                        1        234   
4      713                        0        346   
..     ...                      ...        ...   
482  50284                        1      24261   
483  50675                        0      24451   
484  50676                        1      24451   
485  50788                        0      24507   
486  50789                        1      24507   

                                                                                          post_expansion  
0    If every American would get out of their partisan boxes, this is what every American SHOULD say:...  
1                  I agree with your statement that if there's sufficient evidence, Hunter Biden and Joe  
2                                                  We’re gonna get a mugshot i

In [27]:
NilsThread500merge = NilsThread500merge.merge(expanded_X500_posts_Azure33.add_suffix('_Azure33'), left_on='indexcol', right_on='index_Azure33', how='left').drop(columns=['index_Azure33', 'ascending_comment_index_Azure33', 'thread_id_Azure33'], axis=1)

In [ ]:
#save dataframe to parquet file: 
NilsThread500merge.to_parquet('data/TWON_export_Nils/NilsThread500merge.parquet', index=False, engine='pyarrow')
expanded_X500_posts_Azure33.to_parquet('data/TWON_export_Nils/expanded_X500_posts_Azure33.parquet', index=False, engine='pyarrow')

In [5]:
#load the parquet file:
NilsThread500merge = pd.read_parquet('data/TWON_export_Nils/NilsThread500merge.parquet', engine='pyarrow')
expanded_X500_posts_Azure33 = pd.read_parquet('data/TWON_export_Nils/expanded_X500_posts_Azure33.parquet', engine='pyarrow')

In [28]:
#now try with Azure llama3.1 nf:
#try running the expansion via the UvA Azure cloud:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_id'

#sort data by Time_comment
groupeddata = NilsThread1k[:500].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df = df.loc[df.full_text != '',:]
    df.sort_values(by=['parsed_create_time'], ascending=True).reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.ascending_comment_index.shift(1)
    df.set_index('ascending_comment_index', inplace=True, drop=False)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[row['indexcol'], row['ascending_comment_index'], row['thread_id'], row['full_text']]],
                    columns=['index', 'ascending_comment_index', 'thread_id', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            retry_count = 0
            max_retries = 5
           
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODEL31largeAzureNF,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": SYSTEM_expansion
                                    },
                                    {
                                        "role": "user",
                                        "content": f'"Thread":\n<{df["full_text"][:index].to_list()}>, "Target reply":<<{row["full_text"]}>>',
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                'max_tokens' : 5000,
                                
                            }
                        )  
                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[row['indexcol'], row['ascending_comment_index'], row['thread_id'], data_response["choices"][0]["message"]["content"]]],                                
                            columns=['index', 'ascending_comment_index', 'thread_id', 'post_expansion']
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        retry_after = response.headers.get("Retry-After")
                        error_message = response.json().get("error", {}).get("message", "")
                        retry_after = 30  # Default to 30 seconds if not found

                        # Extract retry time from the error message
                        if "Try again in" in error_message:
                            match = re.search(r"Try again in (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                        elif "Please retry after" in error_message:
                            match = re.search(r"Please retry after (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                                
                        else:
                            retry_after = 30  # Default to 30 seconds if not found
                            print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")

                        print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                        time.sleep(retry_after)

                    #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    #    print(response.text)
                    #    time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 

expanded_X500_posts_Azure31NF0 = pd.concat(chunked_result, ignore_index=True)
print(expanded_X500_posts_Azure31NF0)


  5%|▍         | 11/234 [00:10<04:43,  1.27s/it]

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400}}. Received Model Group=nf-Llama-3.1-70b-instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


 19%|█▉        | 44/234 [00:53<07:58,  2.52s/it]

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400}}. Received Model Group=nf-Llama-3.1-70b-instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


 22%|██▏       | 52/234 [01:04<06:52,  2.27s/it]

...

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400}}. Received Model Group=nf-Llama-3.1-70b-instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


 94%|█████████▎| 219/234 [03:56<00:31,  2.10s/it]

Failed to connect to API. Status code: 400
{"error":{"message":"litellm.BadRequestError: Azure_aiException - {\"error\":{\"message\":\"The response was filtered due to the prompt triggering Microsoft's content management policy. Please modify your prompt and retry.\",\"type\":null,\"param\":\"prompt\",\"code\":\"content_filter\",\"status\":400}}. Received Model Group=nf-Llama-3.1-70b-instruct\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


100%|██████████| 234/234 [04:06<00:00,  1.05s/it]

     index  ascending_comment_index  thread_id  \
0      117                        0         57   
1      118                        1         57   
2      483                        0        234   
3      484                        1        234   
4      713                        0        346   
..     ...                      ...        ...   
483  50284                        1      24261   
484  50675                        0      24451   
485  50676                        1      24451   
486  50788                        0      24507   
487  50789                        1      24507   

                                                                                          post_expansion  
0    If every American would get out of their partisan boxes, this is what every American SHOULD say:...  
1    I agree with the statement that if there's sufficient evidence, Hunter Biden and Joe Biden shoul...  
2                                                  We’re gonna get a mugshot i

In [29]:
NilsThread500merge = NilsThread500merge.merge(expanded_X500_posts_Azure31NF0.add_suffix('_Azure31NF0'), left_on='indexcol', right_on='index_Azure31NF0', how='left').drop(columns=['index_Azure31NF0', 'ascending_comment_index_Azure31NF0', 'thread_id_Azure31NF0'], axis=1)

In [35]:
#save dataframe to parquet file: 
NilsThread500merge.to_parquet('data/TWON_export_Nils/NilsThread500merge.parquet', index=False, engine='pyarrow')
expanded_X500_posts_Azure31NF0.to_parquet('data/TWON_export_Nils/expanded_X500_posts_Azure31NF0.parquet', index=False, engine='pyarrow')

In [ ]:
#inspect errors: row 11 (filtered) and 17 (invalid, code 500) are supposed to be errors
expanded_X500_posts_Azure31NF.iloc[10:20]

,index,ascending_comment_index,thread_id,post_expansion
10,1086,2,527,"@Theo_TJ_Jordan, I appreciate you too, thank you for sharing"
11,1200,0,583,Is there a reality TV show you'll admit to watching?
12,1201,1,583,"""@Mollyploofkins, is the reality TV show you watch '"
13,1276,0,618,You know you all these indictments in Georgia have gotten me thinking about the scale of the cri...
14,1277,1,618,"The plan was to claim it was the will of the people, a movement of"
15,1361,0,658,"Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \..."
16,1362,1,658,@RepAdamSchiff https://t.co/an27n8zNz
17,1943,0,939,Today makes 13 months since your life was stolen. I love you and miss you Christian💔 https://t.c...
18,1944,1,939,"It's been 13 months since Christian's passing, praying for you and your"
19,2315,0,1121,The DOJ is either diverting attention away from Biden family corruption or attempting take out t...


In [32]:
expanded_X500_posts_Azure31NF0.iloc[10:20].post_expansion.to_list()

['@Theo_TJ_Jordan, I appreciate you too, thank you for sharing the link and your thoughts.',
 "Is there a reality TV show you'll admit to watching?",
 '\'@Mollyploofkins, is the reality TV show you watch "Is it cake?"?\'',
 'You know you all these indictments in Georgia have gotten me thinking about the scale of the criminality involved in trying to steal the election for Donald Trump.\n\nWe had lawyers writing papers on how to keep Trump in office, slates of fake electors in several states, people… https://t.co/nxu6g1dZR2',
 'The plan was to claim it was the will of the people, a movement of so many people involved in trying to steal the election for Donald Trump that it would be too many to jail.',
 'Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \n\nThis will put our democracy to a new test:\n\nCan the rule of law be enforced against a former president and current candidate?\n\nFor the sake of our democracy, that answer must be yes.',


In [31]:
NilsThread500merge.iloc[10:20].loc[:, ['full_text', 'ascending_comment_index', 'post_expansion', 'post_expansion_Azure33', 'post_expansion_Azure31NF','post_expansion_Azure31NF0']]

,full_text,ascending_comment_index,post_expansion,post_expansion_Azure33,post_expansion_Azure31NF,post_expansion_Azure31NF0
10,@Theo_TJ_Jordan I appreciate you too. Thank you.,2,"I appreciate you too, Theo, thank you for your kind words and for sharing the post https://t.co/...","@Theo_TJ_Jordan, I appreciate you too, thank you, in","@Theo_TJ_Jordan, I appreciate you too, thank you for sharing","@Theo_TJ_Jordan, I appreciate you too, thank you for sharing the link and your thoughts."
11,Is there a reality TV show you'll admit to watching?,0,Is there a reality TV show you'll admit to watching?,Is there a reality TV show you'll admit to watching?,Is there a reality TV show you'll admit to watching?,Is there a reality TV show you'll admit to watching?
12,@Mollyploofkins Is it cake?!?!,1,"'@Mollyploofkins, when you asked if there's a reality TV show I'll admit to watching, my answer...","'@Mollyploofkins, as for the reality TV show you watch","""@Mollyploofkins, is the reality TV show you watch '","'@Mollyploofkins, is the reality TV show you watch ""Is it cake?""?'"
13,You know you all these indictments in Georgia have gotten me thinking about the scale of the cri...,0,You know you all these indictments in Georgia have gotten me thinking about the scale of the cri...,You know you all these indictments in Georgia have gotten me thinking about the scale of the cri...,You know you all these indictments in Georgia have gotten me thinking about the scale of the cri...,You know you all these indictments in Georgia have gotten me thinking about the scale of the cri...
14,"@TheRealThelmaJ1 Their plan was “it was the will of the people,” a movement of so many “too many...",1,"Their plan was to claim it was the will of the people, framing it as a widespread movement with ...","Their plan was to claim it was the will of the people, framing it as","The plan was to claim it was the will of the people, a movement of","The plan was to claim it was the will of the people, a movement of so many people involved in tr..."
15,"Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \...",0,"Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \...","Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \...","Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \...","Breaking: Trump was just indicted on his most serious charges to date, those concerning Jan 6. \..."
16,@RepAdamSchiff https://t.co/an27n8zNzj,1,"In response to the news about Trump's indictment on serious charges related to Jan 6, @RepAdamSc...","'In response to the indictment of Trump on serious charges concerning Jan 6,",@RepAdamSchiff https://t.co/an27n8zNz,"@RepAdamSchiff, regarding the breaking news that Trump was just indicted on his most serious cha..."
17,Today makes 13 months since your life was stolen. I love you and miss you Christian💔 https://t.c...,0,Today makes 13 months since your life was stolen. I love you and miss you Christian💔 https://t.c...,Today makes 13 months since your life was stolen. I love you and miss you Christian💔 https://t.c...,Today makes 13 months since your life was stolen. I love you and miss you Christian💔 https://t.c...,Today makes 13 months since your life was stolen. I love you and miss you Christian💔 https://t.c...
18,@GiannoCaldwell Praying for you and your family,1,Praying for you and your family as you remember Christian 13 months after their passing.,"It's been 13 months since Christian's passing, and I'm praying for","It's been 13 months since Christian's passing, praying for you and your","It's been 13 months since Christian's passing, praying for you and your family."
19,The DOJ is either diverting attention away from Biden family corruption or attempting take out t...,0,The DOJ is either diverting attention away from Biden family corruption or attempting take out t...,The DOJ

In [ ]:
#it appears some max_tokens is quietly passed restricting output length, so we need to set this to 5000...

below are only notes, analysis ends here for now

In [ ]:
#get similarity scores for claims using GPT4o:
#adjusted request to compare claims only with preceding claims in previous comments, so not preceding claims within the same comment:
#get simscore for posts using GPT4o:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_claim_embeds_exp_explode.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)

for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df = df.loc[df.claims_exp_split != '',:]
    df.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_comment_index']):
            if row['claims_exp_split'] != '':
                retry_count = 0
                max_retries = 10

                while retry_count < max_retries:
                    try:  
                        response = requests.post(
                                url=api_endpoint,
                                headers=headers,
                                json={
                                    'model': MODELgpt4o,
                                    'messages': [
                                        {
                                            "role": "system",
                                            "content": SYSTEM_claim_simscore                        
                                        },
                                        {
                                            "role": "user",
                                            "content": f'"Target claim":<<{row["claims_exp_split"]}>>, "preceding claims":\n<{df.loc[df.ascending_comment_index != row['ascending_comment_index'], ['exp_explode_index', 'claims_exp_split']].iloc[:index].to_json(orient='records')}>'
                                        }
                                    ],
                                    'temperature': temperature_0,  
                                    'seed': SEED
                                }
                            )

                        if response.status_code == 200:
                            data_response = response.json()
                            # Check if 'response' key exists in the JSON response
                            if 'choices' in data_response:
                                content = data_response["choices"][0]["message"]["content"]
                                # Remove code block markers if present
                                content = re.sub(r'```json|```', '', content).strip()
                                try:
                                    content = json.loads(content)   
                                    # Check if content_json is a list or a dictionary
                                    if isinstance(content, dict):
                                        chunked_result.append({
                                            "claim_exp_index": row['exp_explode_index'],
                                            "most_similar_claim_index": content.get('most_similar_claim_index', ''),
                                            'similarity_score': content.get('similarity_score', '')  
                                        })
                                    elif isinstance(content, list):
                                        for item in content:
                                            chunked_result.append({
                                            "claim_exp_index": row['exp_explode_index'],
                                            "most_similar_claim_index": item.get('most_similar_claim_index', ''),
                                            'similarity_score': item.get('similarity_score', '') 
                                            })
                                    else:
                                        print(f"Unexpected content type: {type(content)}")

                                except json.JSONDecodeError as e:
                                    print(f"Error decoding JSON: {e}")
                                    print(data_response)
                            else:
                                print(f"No 'choices' key in response or 'choices' is empty: {content}")                        
                            break  # Exit the retry loop on success  
                        elif response.status_code == 429:
                            retry_count += 1
                            retry_after = response.headers.get("Retry-After")
                            error_message = response.json().get("error", {}).get("message", "")
                            retry_after = 30  # Default to 30 seconds if not found

                            # Extract retry time from the error message
                            if "Try again in" in error_message:
                                match = re.search(r"Try again in (\d+) second", error_message)
                                if match:
                                    try:
                                        retry_after = int(match.group(1))
                                    except (IndexError, ValueError) as e:
                                        print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                        pass
                            elif "Please retry after" in error_message:
                                match = re.search(r"Please retry after (\d+) second", error_message)
                                if match:
                                    try:
                                        retry_after = int(match.group(1))
                                    except (IndexError, ValueError) as e:
                                        print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                        pass
                                    
                                    
                            else:
                                retry_after = 30  # Default to 30 seconds if not found
                                print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")


                            print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                            time.sleep(retry_after)

                        #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        #    print(response.text)
                        #    time.sleep(wait_time)
                        elif response.status_code == 500:
                            retry_count += 1
                            wait_time = 20
                            print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                            print(response.text)
                            time.sleep(wait_time)
                        else:
                            print(f"Failed to connect to API. Status code: {response.status_code}")
                            print(response.text)
                            break
                    except requests.exceptions.RequestException as e:   
                        print(f"Failed to connect to API: {e}")
                        retry_count += 1
                        wait_time = 60
                        print(f"Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)   

                    if retry_count >= max_retries:
                        print(f"Max retries reached for index {index}. Skipping to next item.")
                        break  # Exit the loop if max retries are reached              

# Convert the list of dictionaries to a DataFrame
similarity_claim_exp = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(similarity_claim_exp.head(4))

In [ ]:
#get claims for the expanded posts:

chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm(reddit_ptbod_depth1.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[[row['id'],
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODEL33large,
                            'system': SYSTEM_claim,
                            'prompt': f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["post_expansion_llama33_70b"]}>',
                            'options': options_zero
                            }).json()['response']                       
                ]],
                columns=['id', 'claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

claim_exp_posts = pd.concat(chunked_result, ignore_index=True)
print(claim_exp_posts)

In [ ]:
#now try mining the claims from the original posts with context in one go:
chunked_result2: typing.List[pd.DataFrame] = []
for index, row in tqdm(reddit_ptbod_depth1.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[[row['id'],
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODEL33large,
                            'system': SYSTEM_context_claim,
                            'prompt': f'Check whether your answer strictly adheres to the specified format. \n"Target reply":\n<{row["body"]}\n"Context":\n<{row["thread_title"]}>',
                            'options': options_zero
                            }).json()['response']                       
                ]],
                columns=['id', 'claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

claim_exp_posts = pd.concat(chunked_result2, ignore_index=True)
print(claim_exp_posts)

In [ ]:
#now find the simscore for the expanded claims of on-topic political posts using GPT4o:
#get similarity scores for claims using GPT4o:
#adjusted request to compare claims only with preceding claims in previous comments, so not preceding claims within the same comment:
#get simscore for posts using GPT4o:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_claim_embeds_exp_explode.loc[(YT_claim_embeds_exp_explode.HAS_OPINION_DUMMY==1)&(YT_claim_embeds_exp_explode.topiccode!=0),:].sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)

for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df = df.loc[df.claims_exp_split != '',:]
    df.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_comment_index']):
            if row['claims_exp_split'] != '':
                retry_count = 0
                max_retries = 10

                while retry_count < max_retries:
                    try:  
                        response = requests.post(
                                url=api_endpoint,
                                headers=headers,
                                json={
                                    'model': MODELgpt4o,
                                    'messages': [
                                        {
                                            "role": "system",
                                            "content": SYSTEM_claim_simscore                        
                                        },
                                        {
                                            "role": "user",
                                            "content": f'"Target claim":<<{row["claims_exp_split"]}>>, "preceding claims":\n<{df.loc[df.ascending_comment_index != row['ascending_comment_index'], ['exp_explode_index', 'claims_exp_split']].iloc[:index].to_json(orient='records')}>'
                                        }
                                    ],
                                    'temperature': temperature_0,  
                                    'seed': SEED
                                }
                            )

                        if response.status_code == 200:
                            data_response = response.json()
                            # Check if 'response' key exists in the JSON response
                            if 'choices' in data_response:
                                content = data_response["choices"][0]["message"]["content"]
                                # Remove code block markers if present
                                content = re.sub(r'```json|```', '', content).strip()
                                try:
                                    content = json.loads(content)   
                                    # Check if content_json is a list or a dictionary
                                    if isinstance(content, dict):
                                        chunked_result.append({
                                            "claim_exp_index": row['exp_explode_index'],
                                            "most_similar_pol_exp_claim_index": content.get('most_similar_claim_index', ''),
                                            'pol_exp_claim_similarity_score': content.get('similarity_score', '')  
                                        })
                                    elif isinstance(content, list):
                                        for item in content:
                                            chunked_result.append({
                                            "claim_exp_index": row['exp_explode_index'],
                                            "most_similar_pol_exp_claim_index": item.get('most_similar_claim_index', ''),
                                            'pol_exp_claim_similarity_score': item.get('similarity_score', '') 
                                            })
                                    else:
                                        print(f"Unexpected content type: {type(content)}")

                                except json.JSONDecodeError as e:
                                    print(f"Error decoding JSON: {e}")
                                    print(data_response)
                            else:
                                print(f"No 'choices' key in response or 'choices' is empty: {content}")                        
                            break  # Exit the retry loop on success  
                        elif response.status_code == 429:
                            retry_count += 1
                            retry_after = response.headers.get("Retry-After")
                            error_message = response.json().get("error", {}).get("message", "")
                            retry_after = 30  # Default to 30 seconds if not found

                            # Extract retry time from the error message
                            if "Try again in" in error_message:
                                match = re.search(r"Try again in (\d+) second", error_message)
                                if match:
                                    try:
                                        retry_after = int(match.group(1))
                                    except (IndexError, ValueError) as e:
                                        print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                        pass
                            elif "Please retry after" in error_message:
                                match = re.search(r"Please retry after (\d+) second", error_message)
                                if match:
                                    try:
                                        retry_after = int(match.group(1))
                                    except (IndexError, ValueError) as e:
                                        print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                        pass
                                    
                                    
                            else:
                                retry_after = 30  # Default to 30 seconds if not found
                                print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")


                            print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                            time.sleep(retry_after)

                        #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        #    print(response.text)
                        #    time.sleep(wait_time)
                        elif response.status_code == 500:
                            retry_count += 1
                            wait_time = 20
                            print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                            print(response.text)
                            time.sleep(wait_time)
                        else:
                            print(f"Failed to connect to API. Status code: {response.status_code}")
                            print(response.text)
                            break
                    except requests.exceptions.RequestException as e:   
                        print(f"Failed to connect to API: {e}")
                        retry_count += 1
                        wait_time = 60
                        print(f"Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)   

                    if retry_count >= max_retries:
                        print(f"Max retries reached for index {index}. Skipping to next item.")
                        break  # Exit the loop if max retries are reached              

# Convert the list of dictionaries to a DataFrame
pol_exp_similarity_claim_exp = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(pol_exp_similarity_claim_exp.head(4))

In [ ]:
#try asking for a similarity score instead:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.claim_index.shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append({
                "claim_index": row['claim_index'],
                "most_similar_claim_index": '',
                'similarity_score': ''  
                    })
                
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            if row['claims'] == '[]':
                chunked_result.append({
                    "claim_index": row['claim_index'],
                    "most_similar_claim_index": '',
                    'similarity_score': ''  
                    })
            else:
                try: 
                    response = requests.post(
                                    'https://inf.cl.uni-trier.de/',
                                    json={
                                        'model': MODELsmall,
                                        'system': SYSTEM_claim_simscore,
                                        'prompt': f'Reason about your respons, but respond with nothing else than the JSON. "Target claim":<<{row["claims"]}>>, "preceding claims and indices":\n<{df.loc[:,['claim_index', 'claims']][:threadset].to_json(orient='records')}>',
                                        'options': options_zero
                                        }).json()
                    # Check if 'response' key exists in the JSON response
                    if 'response' in response:
                        for claim in json.loads(response['response']):
                            chunked_result.append({
                                "claim_index": row['claim_index'],
                                "most_similar_claim_index": claim.get('most_similar_claim_index', ''),
                                'similarity_score': claim.get('similarity_score', '')  
                        })
                    else:
                        print(f"Key 'response' not found in the response: {response}")                      
    
                except json.JSONDecodeError:
                    print("invalid json response, skipping to next batch")

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(result_df.head(4))